# Script of knot sampling with Cobaya that reconstructs P_R(k) from J-PAS LRG mono/quadrupole galaxy PS data combining all z-bins #

In [17]:
#Sections with * indicate that they should be modify if catalogue specifications are changed

#Sections with ** indicate that the path or output filenames must be modified according to the desired specifications

# Needed packages**

In [18]:
#Packages needed
import cobaya
import sys
#Specify in this path where your CAMB is installed**
sys.path.append('/Users/guillermo/Desktop/code/CAMB')
import camb
import numpy as np
import sympy
import math
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
import scipy.integrate as integrate

# Fiducial cosmology: parameters #

In [19]:
#Cosmological constants:
c = 2.99792458E5;   H = 1/(c/100);

#Parameteres that won't be sampled:
OmegakJPAS = 0;    gamma = 0.545;     kPivot = 0.05 #Mpc{-1}

#Baseline fixed cosmology fromPlanck 2018-values TT+TE+EE+lowE+lensing. Neutrinos are assumed as 0 mass:
AsJPAS = 2.09052E-9; nsJPAS = 0.9646; tauJPAS = 0.0544; mnuJPAS = 0.0; nmuJPAS = 3.046;

#Fiducial for the baseline cosmological parameters that are going to be sampled:
hJPAS = 0.6737
OmegabJPASh2 = 0.02237
OmegaCDMJPASh2 = 0.1200

#Indirect cosmological parameters:
H0JPAS = hJPAS*100
OmegabJPAS = OmegabJPASh2/hJPAS**2;   OmegaCDMJPAS = OmegaCDMJPASh2/hJPAS**2;

# More cosmological parameters* #

In [20]:
#Fiducial cosmology functions and constants (including FoG parameter sigmap)

#Omega matter:
OmegamFid = 0.3136789606771906

#Cosmological functions at all redshift bins, from the J-PAS LRG bins (i.e. z=0.1,0.3,0.5,0.7,0.9)
EzFid = np.array([1.05063206, 1.17280592, 1.32097976, 1.49245629, 1.68459046])
XiFid = np.array([292.61681198, 833.57665297, 1315.76329791, 1743.02284215, 2121.25261255])
DAFid = np.array([266.01528362, 641.21280998, 877.17553194, 1025.3075542, 1116.44874345])
sigmapFid = np.array([5.60682192, 5.04305103, 4.54769682, 4.11993136, 3.75297894])

In [21]:
#Bias for the J-PAS LRG. Just divide the bias_0 (1.70) by the growth factor D(z):
bJPAS = np.array([1.70/0.94847114, 1.70/0.85308379, 1.70/0.76926768, 1.70/0.69688361, 1.70/0.63478618 ])

In [22]:
#Area of the sky, obtained from the number of filters in the tray and the years of observation.

#Choose the tray filter strategy*:
AreaPerYear2TrayFilters = 900 #In sq degrees per year
#AreaPerYear4TrayFilters = 450 #In sq degrees per year

#Choose the observation time*:
ObservationTime = 5 #in years
#ObservationTime = 2.5 #in years

#Calculate the area (in sq. degrees) and fraction of the sky:
Asky = AreaPerYear2TrayFilters*ObservationTime
fsky = Asky/(4*np.pi*(180/np.pi)**2)

In [23]:
#Power law primordial power spectrum. Scale k must be in h units.
def PrimordialPowerLaw(As,ns,k):
    return As*(k/(0.05/hJPAS))**(ns-1)

In [24]:
#Power law primordial power spectrum without h units.
def PrimordialPowerLawSinh(As,ns,k):
    return As*(k/(0.05))**(ns-1)

# k and z binning #

In [25]:
#Scale and redshift bins arrays limits and steps. 

#Extended arrays are also calculated.

#k array limits, in h Mpc{-1} units: 
khminKhArrayJPASComplete = 0.001;   khmaxKhArrayJPASComplete = 2.4900;  stepKhArrayJPASComplete = 0.025;

#k binning, complete and in a reduced scaleset. In h Mpc{-1}:
KhArrayJPASComplete = np.exp(np.arange(math.log(khminKhArrayJPASComplete), math.log(khmaxKhArrayJPASComplete), stepKhArrayJPASComplete) )
KhArrayJPAS = KhArrayJPASComplete[range(120,212)]

#k binning on lower and upper limits
KhArrayJPASUpper = np.zeros(len(KhArrayJPAS)); KhArrayJPASLower = np.zeros(len(KhArrayJPAS));

for i in range(0, len(KhArrayJPAS)-1):
    KhArrayJPASUpper[i] = KhArrayJPAS[i] + (KhArrayJPAS[i+1]-KhArrayJPAS[i])/2;
    KhArrayJPASLower[i] = KhArrayJPAS[i] - (KhArrayJPAS[i+1]-KhArrayJPAS[i])/2;

#The last element of KUpper or KLower can be problematic. We copy the last element into the penultimate position
KhArrayJPASUpper[-1] = KhArrayJPASUpper[-2];  KhArrayJPASLower[-1] = KhArrayJPASLower[-2];

#z binning for high-z
zJPASmin = 0.1;   zJPASmax = 0.9;   stepzJPAS = 0.2;

#Original z-bin array
zJPASPrevious = np.arange(zJPASmin, zJPASmax+stepzJPAS/2, stepzJPAS)

#Element z that can be included to the array (for computing quantities at z=0)
zJPASAdditional = np.array([0])

#Binning including all lower and upper z-bins limits
zJPAS = np.arange(zJPASmin-stepzJPAS/2, zJPASmax+0.01+stepzJPAS/2, stepzJPAS/2)

#Positions of upper and lower limits of the z-bins in the z array
positions_Upper = [2, 4, 6, 8, 10]
positions_Lower = [0, 2, 4, 6, 8]

 # P(k) data and densities reading** #

In [26]:
# Define a class to read the simulated data (Pk data, densities) and if neccesary for doing checks,
# the transfer function and the seed. The path must be specified as input, and the end of this cell.


def read_data(path_to_data):
    data = {}

    #Read the proper data with the proper specifications :
    Simulated_pk_filename_z01 = path_to_data+'/LRG P_g0 forecasted data/JPAS_ForecastData_LRG_z0.1_LO3_LowDeltaZ_g12_5years.dat'
    Simulated_pk_filename_z03 = path_to_data+'/LRG P_g0 forecasted data/JPAS_ForecastData_LRG_z0.3_LO3_LowDeltaZ_g12_5years.dat'
    Simulated_pk_filename_z05 = path_to_data+'/LRG P_g0 forecasted data/JPAS_ForecastData_LRG_z0.5_LO3_LowDeltaZ_g12_5years.dat'
    Simulated_pk_filename_z07 = path_to_data+'/LRG P_g0 forecasted data/JPAS_ForecastData_LRG_z0.7_LO3_LowDeltaZ_g12_5years.dat'
    Simulated_pk_filename_z09 = path_to_data+'/LRG P_g0 forecasted data/JPAS_ForecastData_LRG_z0.9_LO3_LowDeltaZ_g12_5years.dat'
    Simulated_densities = path_to_data+'/Densities Mini J-PAS Antonio Maroto/nlrg_12_lowdeltaz_per_bin_no_header.tex'

    data['pk0z'] = np.zeros((len(zJPASPrevious), len(KhArrayJPAS))) #To allocate the monopole
    data['pk2z'] = np.zeros((len(zJPASPrevious), len(KhArrayJPAS))) #To allocate the quadrupole
    data['ndz'] = np.zeros(len(zJPASPrevious)) #Allocate densities
    data['deltaz'] = np.zeros(len(zJPASPrevious)) #Allocate photometric errors delta_z
    data['vs'] = np.zeros(len(KhArrayJPAS)) #Allocate a seed, if needed
  
    with open(Simulated_pk_filename_z01) as file:
        for i in range(len(KhArrayJPAS)):
            line = file.readline().split()
            data['pk0z'][0][i] = float(line[7])    #In the data files, the realized monopole/quadrupole is in line 7/10
            data['pk2z'][0][i] = float(line[10])
            data['vs'][i] = float(line[6])

    with open(Simulated_pk_filename_z03) as file:
        for i in range(len(KhArrayJPAS)):
            line = file.readline().split()
            data['pk0z'][1][i] = float(line[7])
            data['pk2z'][1][i] = float(line[10])

    with open(Simulated_pk_filename_z05) as file:
        for i in range(len(KhArrayJPAS)):
            line = file.readline().split()
            data['pk0z'][2][i] = float(line[7])
            data['pk2z'][2][i] = float(line[10])

    with open(Simulated_pk_filename_z07) as file:
        for i in range(len(KhArrayJPAS)):
            line = file.readline().split()
            data['pk0z'][3][i] = float(line[7])
            data['pk2z'][3][i] = float(line[10])
            
    with open(Simulated_pk_filename_z09) as file:
        for i in range(len(KhArrayJPAS)):
            line = file.readline().split()
            data['pk0z'][4][i] = float(line[7])
            data['pk2z'][4][i] = float(line[10])

    with open(Simulated_densities) as file:
        for i in range(len(zJPASPrevious)):
            line = file.readline().split()
            data['ndz'][i] = float(line[1])  #Densities in line 1
            data['deltaz'][i] = float(line[2]) #Photometric errors in line 2

                          
    return data

# Read data is converted in the dictionary 'data'

#Specify the path of the datafile folder:
#data = read_data('/gpfs/users/martinezg/J-PAS_Forecast_Data/')
data = read_data('/Users/guillermo/Desktop/J-PAS_Forecast_Data/')
data.keys()

#Create an array that includes the photometric error in the extended z-bin elements: 
DeltazLRGJPAS = np.zeros(len(zJPAS))

for i in range(len(zJPASPrevious)):
    DeltazLRGJPAS[2*i + 1] = data['deltaz'][i]

# Classes to interface with Cobaya* ** #

In [27]:
#The classes to interface with Cobaya are created.

#A cobaya theory NodesInPrimordialPk and a cobaya external likelihood Pklike classes are created
from cobaya.theory import Theory
from cobaya.likelihood import Likelihood

In [28]:
#Class of the theory, with the PPS modification including the knots. 
#This is an example of a 4 knot case.

class NodesInPrimordialPk(Theory):

    def initialize(self): #Initialize self with the k-array.
        self.ks = KhArrayJPAS

    #Definition of knot parameters to be sampled.
    def calculate(self, state, want_derived=True, **params_values_dict):

        #This part of the code orders the variables x in order to avoid the label switching problem.    
        number_nodes = 4
        number_nodes_red = number_nodes-2

        megacube = np.zeros(number_nodes)
        
        megacube[0] = params_values_dict['x1']

        megacube[1] = megacube[0] + (1 - megacube[0]) * (1 - (1 - params_values_dict['x2']) ** (1 /(number_nodes_red+1-(2-1)) ))
        megacube[2] = megacube[1] + (1 - megacube[1]) * (1 - (1 - params_values_dict['x3']) ** (1 /(number_nodes_red+1-(3-1)) ))
    
        megacube[3] = params_values_dict['x4']

        #Here we define que k (named 'x') and PPS nodes (named 'y').

        nodes_logk = [(np.log(KhArrayJPAS[-1])-np.log(KhArrayJPAS[0]) ) * megacube[0] + np.log(KhArrayJPAS[0]), 
                      (np.log(KhArrayJPAS[-1])-np.log(KhArrayJPAS[0]) ) * megacube[1] + np.log(KhArrayJPAS[0]),
                      (np.log(KhArrayJPAS[-1])-np.log(KhArrayJPAS[0]) ) * megacube[2] + np.log(KhArrayJPAS[0]),
                      (np.log(KhArrayJPAS[-1])-np.log(KhArrayJPAS[0]) ) * megacube[3] + np.log(KhArrayJPAS[0])] 
        
        nodes_logPPS = [params_values_dict['y1'], params_values_dict['y2'],params_values_dict['y3'],params_values_dict['y4']]



        #nodes_k and nodes_PPS are linearly interpolated in log space. For outer values, we extraplote.
        NodesInterpFunc_nodes = interp1d(nodes_logk, nodes_logPPS,
        kind='linear', fill_value='extrapolate')

        
        #We construct a modified PPS(k) evaluated at our nodes and interpolated, evaluated at our k-array.
        #The units must be without h in the limits of k
        state['primordial_scalar_pk'] = {'kmin': KhArrayJPAS[0]*hJPAS, 'kmax': KhArrayJPAS[-1]*hJPAS,
                                            'Pk': np.exp(NodesInterpFunc_nodes(np.log(KhArrayJPAS))), 'log_regular': True}
        
        
    #Name the modified primordial power spectrum as 'primordial_scalar_pk' to be called in Cobaya.
    def get_primordial_scalar_pk(self):
        return self.current_state['primordial_scalar_pk']
        
    #Function that contains the parameters to be sampled.
    def get_can_support_params(self):
        return ['x1', 'x2', 'x3', 'x4', 'y1', 'y2','y3', 'y4']



In [29]:
#Class incorporating the model (quadrupole galaxy power spectrum) and the likelihood. 

class Pklike(Likelihood): 

    def initialize(self):  

        #Path in which the data is. We call read_data with this path**.
        #self.data = read_data('/gpfs/users/martinezg/J-PAS_Forecast_Data/')
        self.data = read_data('/Users/guillermo/Desktop/J-PAS_Forecast_Data/')

        #k-array
        self.ks = KhArrayJPAS
        
        #z-bins
        self.z_win = zJPAS

    
    def get_requirements(self): #Cobaya functions that we might use.
        
        return {'omegam': None,                
                'Pk_interpolator': {'z': self.z_win, 'k_max': 10, 'nonlinear': False, 'vars_pairs': ([['delta_tot', 'delta_tot']])},
                'comoving_radial_distance': {'z': self.z_win},
                'angular_diameter_distance': {'z': self.z_win},
                'Hubble': {'z': self.z_win, 'units': 'km/s/Mpc'},
                'sigma8_z': {'z': self.z_win}, 'fsigma8': {'z': self.z_win},
                 #'fsigma8': {'z': self.z_win, 'units': None},
                'CAMBdata': None}

        
    #Definition of the multipoles: monopole and quadrupole. It will return:
        #The monopole evaluated at the z-bins and the k-array.
        #The monopole covariance evaluated at the z-bins and the k-array.  
    
        #The quadrupole evaluated at the z-bins and the k-array.
        #The quadrupole covariance evaluated at the z-bins and the k-array.  
    
    def multipoles(self, **params_dic):
        
        #Options for print with enough decimal precision, if needed
        #np.set_printoptions(precision=24, suppress=True)

        #CAMB results within Cobaya
        resultsCobaya = self.provider.get_CAMBdata() 

        #This is the modified primordial power spectrum P(k) evaluated at KArray.
        primordialCobaya = self.provider.get_primordial_scalar_pk()
        
        #Construction of Pmatter(k) calling Cobaya:
        pkCobaya = self.provider.get_Pk_interpolator(('delta_tot', 'delta_tot'), nonlinear=False) 
        
        # All functions and variables to compute the Kaiser model. It reads the cosmology from info (below)
        
        #Cosmological parameters from CAMB
        Omegam = self.provider.get_param('omegam')  

        #Cosmological functions
        Ez = np.sqrt( Omegam*(1+self.z_win)**3+(1-Omegam) ); 
        HJPAS = H * Ez
        f = (Omegam*(1+self.z_win)**3*1/(Ez**2))**gamma
        Xi = self.provider.get_comoving_radial_distance(self.z_win)*hJPAS; #CAMB is called here

        DA = Xi/(1+self.z_win);
        
        # Photometric factor
        sigmar = DeltazLRGJPAS*(1+self.z_win)/HJPAS

        # Fingers of God effect at proper z-bin in the fiducial
        def FFog(mu,k,iz):
            return 1/(1+(f[2*iz-1]*k*mu*sigmapFid[iz-1])**2)
      
        # AP effect
        def FactorAP(iz):
            return DAFid[iz-1]**2*Ez[2*iz-1]/( DA[2*iz-1]**2*EzFid[iz-1] )

        def Q(mu,iz):
            return ((Ez[2*iz-1]**2*Xi[2*iz-1]**2*mu**2-EzFid[iz-1]**2*XiFid[iz-1]**2*(mu**2-1))**0.5/(EzFid[iz-1]*Xi[2*iz-1]))

        def muObs(mu,iz):
            return mu*Ez[2*iz-1]/(EzFid[iz-1]*Q(mu,iz))

        def kObs(mu,k,iz):
            return Q(mu,iz)*k

        #Galaxy Power spectrum (mu,k) with AP and FoG. The Pmatter of Cobaya must be evaluated at k without h units
        def Pg(mu,k,iz):
            return FactorAP(iz)*FFog(muObs(mu,iz),kObs(mu,k,iz),iz)*(bJPAS[iz-1]+f[2*iz-1]*muObs(mu,iz)**2)**2 * (   hJPAS**3*(   pkCobaya.P(self.z_win[2*iz-1],kObs(mu,k,iz)*hJPAS)   )   ) *np.exp(-(k*mu*sigmar[2*iz-1])**2)
        


        #Monopole and quadrupole galaxy power spectra

        #Trapezoid rule with 2000 steps for computing the multipoles:

        def Pgmonopole(k,iz):
            mu = np.arange(-1, 1, 1/1000)
            return 1/2 * integrate.trapz(Pg(mu, k, iz), mu)
                    
        def Pgquadrupole(k,iz):
            mu = np.arange(-1, 1, 1/1000)
            return 5/2 * integrate.trapz(Pg(mu, k, iz) * 0.5 * (3*mu**2-1), mu)

       
        #We evaluate both monopole and quadrupole power spectra in our k-array and z-bins
        PgmonopoleValues = np.zeros((len(zJPASPrevious), len(KhArrayJPAS)))
        PgquadrupoleValues = np.zeros((len(zJPASPrevious), len(KhArrayJPAS)))


        for j in range(len(zJPASPrevious)):
            for i in range(len(KhArrayJPAS)):
                PgmonopoleValues[j, i] = Pgmonopole(KhArrayJPAS[i], j+1)
                PgquadrupoleValues[j, i] = Pgquadrupole(KhArrayJPAS[i], j+1)

                 
        #Covariance matrixes

        #Angular distance for z upper and lower bins from CAMB
        XiZaLower = self.provider.get_comoving_radial_distance(self.z_win[positions_Lower])*hJPAS
        XiZaUpper = self.provider.get_comoving_radial_distance(self.z_win[positions_Upper])*hJPAS
        
        #Definition of the volume between redshift bins
        Vol = 4*np.pi*fsky/3*(XiZaUpper**3-XiZaLower**3)
        
        
        #Number of modes. It depends of ksup and kinf corresponding to kupper y klower
        def Nk(ksup,kinf,iz):
            return Vol[iz-1] * (4*np.pi/3*(ksup**3-kinf**3))/((2*np.pi)**3)

        
        #Number of nodes evaluated at our arrays.
        NkEvaluated = np.zeros((len(zJPASPrevious), len(KhArrayJPASUpper)))

        for j in range(len(zJPASPrevious)):
            for i in range(len(KhArrayJPASUpper)):     
                NkEvaluated[j, i] = Nk(KhArrayJPASUpper[i], KhArrayJPASLower[i], j+1)



        #We compute the value of the monopole Covariance Matrix evaluated at the k-arrays for all the bins:

        CovMonopoleEvaluatedz01 = 2 *(PgmonopoleValues[0] + 1/self.data['ndz'][0])**2 / NkEvaluated[0]
        CovMonopoleEvaluatedz03 = 2 *(PgmonopoleValues[1] + 1/self.data['ndz'][1])**2 / NkEvaluated[1]
        CovMonopoleEvaluatedz05 = 2 *(PgmonopoleValues[2] + 1/self.data['ndz'][2])**2 / NkEvaluated[2]
        CovMonopoleEvaluatedz07 = 2 *(PgmonopoleValues[3] + 1/self.data['ndz'][3])**2 / NkEvaluated[3]
        CovMonopoleEvaluatedz09 = 2 *(PgmonopoleValues[4] + 1/self.data['ndz'][4])**2 / NkEvaluated[4]

        
        CovQuadrupoleEvaluatedz01 = 2 *(PgquadrupoleValues[0] + 1/self.data['ndz'][0])**2 / NkEvaluated[0]
        CovQuadrupoleEvaluatedz03 = 2 *(PgquadrupoleValues[1] + 1/self.data['ndz'][1])**2 / NkEvaluated[1]
        CovQuadrupoleEvaluatedz05 = 2 *(PgquadrupoleValues[2] + 1/self.data['ndz'][2])**2 / NkEvaluated[2]
        CovQuadrupoleEvaluatedz07 = 2 *(PgquadrupoleValues[3] + 1/self.data['ndz'][3])**2 / NkEvaluated[3]
        CovQuadrupoleEvaluatedz09 = 2 *(PgquadrupoleValues[4] + 1/self.data['ndz'][4])**2 / NkEvaluated[4]

        #The function multipoles returns the monopole values for each z-bin, its covariance, and the same with the quadrupole.
        return PgmonopoleValues[0], PgmonopoleValues[1], PgmonopoleValues[2], PgmonopoleValues[3], PgmonopoleValues[4], CovMonopoleEvaluatedz01, CovMonopoleEvaluatedz03, CovMonopoleEvaluatedz05, CovMonopoleEvaluatedz07, CovMonopoleEvaluatedz09, PgquadrupoleValues[0], PgquadrupoleValues[1], PgquadrupoleValues[2], PgquadrupoleValues[3], PgquadrupoleValues[4], CovQuadrupoleEvaluatedz01, CovQuadrupoleEvaluatedz03, CovQuadrupoleEvaluatedz05, CovQuadrupoleEvaluatedz07, CovQuadrupoleEvaluatedz09


    #Likelihood calculation. We define the likelihood.
    
    def logp(self, **params_values):  
        
        
        #For allocating the multipoles values and their cov values.

        #For the monopole:
        PmonopoleBined = np.zeros((len(zJPASPrevious), len(self.ks)))
        CovMonopoleBined = np.zeros((len(zJPASPrevious), len(self.ks)))

        #For the quadrupole:
        PquadrupoleBined = np.zeros((len(zJPASPrevious), len(self.ks)))
        CovQuadrupoleBined = np.zeros((len(zJPASPrevious), len(self.ks)))

        #PmonopoleBined, CovMonopoleBined, PquadrupoleBined and CovQuadrupoleBined are equal
        #to the values given by the self.multipoles function:
        PmonopoleBined[0, :len(self.ks)], PmonopoleBined[1, :len(self.ks)], PmonopoleBined[2, :len(self.ks)], PmonopoleBined[3, :len(self.ks)], PmonopoleBined[4, :len(self.ks)], CovMonopoleBined[0, :len(self.ks)], CovMonopoleBined[1, :len(self.ks)], CovMonopoleBined[2, :len(self.ks)], CovMonopoleBined[3, :len(self.ks)], CovMonopoleBined[4, :len(self.ks)], PquadrupoleBined[0, :len(self.ks)], PquadrupoleBined[1, :len(self.ks)], PquadrupoleBined[2, :len(self.ks)], PquadrupoleBined[3, :len(self.ks)], PquadrupoleBined[4, :len(self.ks)], CovQuadrupoleBined[0, :len(self.ks)], CovQuadrupoleBined[1, :len(self.ks)], CovQuadrupoleBined[2, :len(self.ks)], CovQuadrupoleBined[3, :len(self.ks)], CovQuadrupoleBined[4, :len(self.ks)] = self.multipoles(**params_values)
        

        #Construction of the likelihood like a chi^2 with the log of determinant term
        lnlike = 0.0

        #Here the inclusion of the quadrupole and different z-bins can be done:
        for i in range(len(KhArrayJPAS)):
            #For all bins (0-4) and for both monopole and quadrupole.
            lnlike = lnlike + ((PmonopoleBined[0][i] - data['pk0z'][0][i])**2 *1/CovMonopoleBined[0][i] + np.log(CovMonopoleBined[0][i])) +  ((PmonopoleBined[1][i] - data['pk0z'][1][i])**2 *1/CovMonopoleBined[1][i] + np.log(CovMonopoleBined[1][i])) + ((PmonopoleBined[2][i] - data['pk0z'][2][i])**2 *1/CovMonopoleBined[2][i] + np.log(CovMonopoleBined[2][i])) + ((PmonopoleBined[3][i] - data['pk0z'][3][i])**2 *1/CovMonopoleBined[3][i] + np.log(CovMonopoleBined[3][i])) + ((PmonopoleBined[4][i] - data['pk0z'][4][i])**2 *1/CovMonopoleBined[4][i] + np.log(CovMonopoleBined[4][i])) + ((PquadrupoleBined[0][i] - data['pk2z'][0][i])**2 *1/CovQuadrupoleBined[0][i] + np.log(CovQuadrupoleBined[0][i])) +  ((PquadrupoleBined[1][i] - data['pk2z'][1][i])**2 *1/CovQuadrupoleBined[1][i] + np.log(CovQuadrupoleBined[1][i])) + ((PquadrupoleBined[2][i] - data['pk2z'][2][i])**2 *1/CovQuadrupoleBined[2][i] + np.log(CovQuadrupoleBined[2][i])) + ((PquadrupoleBined[3][i] - data['pk2z'][3][i])**2 *1/CovQuadrupoleBined[3][i] + np.log(CovQuadrupoleBined[3][i])) + ((PquadrupoleBined[4][i] - data['pk2z'][4][i])**2 *1/CovQuadrupoleBined[4][i] + np.log(CovQuadrupoleBined[4][i]))
            
            #For all bins (0-4) and for only the monopole.
            #lnlike = lnlike + ((PmonopoleBined[0][i] - data['pk0z'][0][i])**2 *1/CovMonopoleBined[0][i] + np.log(CovMonopoleBined[0][i])) +  ((PmonopoleBined[1][i] - data['pk0z'][1][i])**2 *1/CovMonopoleBined[1][i] + np.log(CovMonopoleBined[1][i])) + ((PmonopoleBined[2][i] - data['pk0z'][2][i])**2 *1/CovMonopoleBined[2][i] + np.log(CovMonopoleBined[2][i])) + ((PmonopoleBined[3][i] - data['pk0z'][3][i])**2 *1/CovMonopoleBined[3][i] + np.log(CovMonopoleBined[3][i])) + ((PmonopoleBined[4][i] - data['pk0z'][4][i])**2 *1/CovMonopoleBined[4][i] + np.log(CovMonopoleBined[4][i]))
        
            #For the first and fifth bins (0 and 4) of the monopole.
            #lnlike = lnlike + ((PmonopoleBined[0][i] - data['pk0z'][0][i])**2 *1/CovMonopoleBined[0][i] + np.log(CovMonopoleBined[0][i])) +  + ((PmonopoleBined[4][i] - data['pk0z'][4][i])**2 *1/CovMonopoleBined[4][i] + np.log(CovMonopoleBined[4][i]))

        return -lnlike/2

# Planck priors (correlated) #

In [30]:
#Creation of the multivariate gaussian that will be used as prior

#Mean values of the Multivariate gaussian for each dimension
mean_vector = [OmegabJPASh2, OmegaCDMJPASh2, H0JPAS]

# Covariance matrix for Planck TTTEEE low l low E lensing.
# Order: Omegabh2, Omegach2 and H0
PlanckDR3_base_plikHM_TTTEEE_lowl_lowE_lensing_covMatrix = np.array([
    [2.12418149e-08, -9.03204492e-08, 5.50402459e-05],
    [-9.03204492e-08, 1.38810729e-06, -6.02340614e-04],
    [5.50402459e-05, -6.02340614e-04, 2.86660525e-01]
])

#Components of the manual 3D guassian to be created:

#Determinant:
determinant = np.linalg.det(PlanckDR3_base_plikHM_TTTEEE_lowl_lowE_lensing_covMatrix)

#Inverse matrix:
if determinant != 0:
    inverse_matrix = np.linalg.inv(PlanckDR3_base_plikHM_TTTEEE_lowl_lowE_lensing_covMatrix)
else:
    print("The matrix is singular and does not have an inverse.")

#Residuals Vector:
def ResidualsVector(ombh2, omch2, H0):
    return np.array([ombh2-OmegabJPASh2,omch2-OmegaCDMJPASh2,H0-H0JPAS])

#Construction of the normalized pdf (we include the log the the logpriors and the -1/2 factor):
def multivariate_gaussian_pdf(ombh2, omch2, H0):
    return np.log((2*np.pi)**(-3/2)*determinant**(-1/2)*np.exp(-1/2* np.dot(np.transpose(ResidualsVector(ombh2, omch2, H0)),np.dot(inverse_matrix,ResidualsVector(ombh2, omch2, H0)))))

# Cobaya Info** #

In [31]:
# Dictionary given to Cobaya. We need to specify:
    #Likelihood (class including the likelihood and the model).
    #Theory class (in which we modify the PPS to have knots).
    #Parameters (fixed and to be sampled).
    #Sampler to be used and specifications.
    #Output where save the output.
    #Afitional options as debugind or resuming chains.


info = {'debug': False,                        #Allow to debug
        'likelihood': {'jpass': Pklike},       #Link likelihood with the previously defined clss
        'theory': {'camb': {"external_primordial_pk": True},
                   'my_pk': NodesInPrimordialPk},      #We include the primordial Pk with nodes in the theory class
       'params': {
           
        # Fixed cosmological parameters. We include the extremal knots in this category.
        'tau': tauJPAS, 'mnu': mnuJPAS, 'nnu': nmuJPAS,
        'x1': 0.0,
        'x4': 1.0,
           
        #Also it seems to be neccesary to include the cosmological parameters, although later they will be sampled.
        'ombh2': OmegabJPASh2, 'omch2': OmegaCDMJPASh2, 'H0': H0JPAS,
           
        #Parameters to be sampled. Knot parameters with flat priors:
        'y1': {'prior': {'min': -23, 'max': -19}, 'ref': -20, 'latex': 'y_1'},
        'y2': {'prior': {'min': -23, 'max': -19}, 'ref': -20, 'latex': 'y_2'},
        'y3': {'prior': {'min': -23, 'max': -19}, 'ref': -20, 'latex': 'y_3'},
        'y4': {'prior': {'min': -23, 'max': -19}, 'ref': -20, 'latex': 'y_4'},
        'x2': {'prior': {'min': 0.0, 'max': 1.0}, 'ref': 0.5, 'latex': 'x_2'},
        'x3': {'prior': {'min': 0.0, 'max': 1.0}, 'ref': 0.5, 'latex': 'x_3'}, 
        
        # Cosmological parameters to be sampled. Loc and scale are the mean value and the st deviation for a guassian prior:
        'ombh2': {'prior': {'dist': 'norm', 'loc': OmegabJPASh2, 'scale': 0.00015}, 'latex': 'Omega_bh^2'},
        'omch2': {'prior': {'dist': 'norm', 'loc': OmegaCDMJPASh2, 'scale': 0.0012}, 'latex': 'Omega_ch^2'},
        'H0': {'prior': {'dist': 'norm', 'loc': H0JPAS, 'scale': 0.54}, 'latex': 'H_0'}},

        #Here the setting of the sampler are chosen. We employ PolyChord. 
        #The nlive by default  equal to 25 * params to be sampled (i.g. 225).
        "sampler": {"polychord":
                        {"precision_criterion": 5e-3}
                    }
           }

#Prior imposed to the sampled cosmological parameters. We employ correlated posteriors according to Planck 2018 TTTEEE low l low E lensing
#as defined above:
info["prior"] = {"Multivariate": lambda ombh2, omch2, H0: multivariate_gaussian_pdf(ombh2, omch2, H0)} 

#Path for output folder and name of the directory**

#info["output"] = "/gpfs/users/martinezg/OutputCobaya_JPAS_LRG/AllBins/LO1_LowDeltaZ_g12_5years_4Nodes_Results"
info["output"] = "/Users/guillermo/Desktop/ProofGitHub/OutputCobaya_JPAS_LRG/AllBins/LO1_MonoAndQuadru_LowDeltaZ_g12_5years_4Nodes_Results"

# Run Cobaya #

In [1]:
#Execute Cobaya:

from cobaya import run
updated_info, sampler = run(info)